# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [6]:
!mkdir data

mkdir: data: File exists


In [7]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31427    0 31427    0     0  96676      0 --:--:-- --:--:-- --:--:-- 96698


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0   182k      0 --:--:-- --:--:-- --:--:--  183k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

[nltk_data] Downloading package punkt to /Users/smajeti/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/smajeti/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/smajeti/aim/bootcamp/aim-smajeti/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/smajeti/aim/bootcamp/aim-smajeti/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/smajeti/aim/bootcamp/aim-smajeti/AIE5/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 70)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 70)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### Answer #1:

* SingleHopSpecificQuerySynthesizer:
requires retrieving information from a single document to a straightforward question that does not need any interpretation of data.
* MultiHopAbstractQuerySynthesizer:
requires retrieval from multiple sources of information, likely from different sources and would need to interpret the data to answer
* MultiHopSpecificQuerySynthesizer:
requires retrieval from multiple sources but can answer without understanding the overall context, data.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [16]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wut is EleutherAI?,[Gullibility is the biggest unsolved problem C...,EleutherAI is one of the organizations that ha...,single_hop_specifc_query_synthesizer
1,Wut iz the relashunship between gullibility in...,[that they haven’t shared yet. Vibes Based Dev...,Gullibility in language models is seen as a si...,single_hop_specifc_query_synthesizer
2,Wut did we figger out about AI in 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,how use homebrew for run llama 2?,[issues in a way that’s surprisingly easy to f...,You can run Llama 2 on your own Mac using LLM ...,single_hop_specifc_query_synthesizer
4,What are the notable features of Claude 3.5 So...,[Prompt driven app generation is a commodity a...,"Claude 3.5 Sonnet, launched in June, is a mode...",single_hop_specifc_query_synthesizer
5,How have efficiency improvements and universal...,[<1-hop>\n\nPrompt driven app generation is a ...,Efficiency improvements and universal access t...,multi_hop_abstract_query_synthesizer
6,How have efficiency improvements and universal...,[<1-hop>\n\nPrompt driven app generation is a ...,Efficiency improvements and universal access t...,multi_hop_abstract_query_synthesizer
7,How have advancements in AI agents and efficie...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI agents and efficiency have ...,multi_hop_abstract_query_synthesizer
8,How does DeepSeek v3's use of synthetic traini...,[<1-hop>\n\nreasoning patterns. Another common...,"DeepSeek v3 utilizes synthetic training data, ...",multi_hop_specific_query_synthesizer
9,How did the advancements in GPT-4 and other LL...,[<1-hop>\n\nyour camera feed with the model an...,"In 2024, advancements in GPT-4 and other large...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [18]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What role has Meta played in the development o...,[Code may be the best application The ethics o...,"Meta released Llama, which allowed users to ru...",single_hop_specifc_query_synthesizer
1,What are the main challenges associated with L...,[Based Development As a computer scientist and...,The main challenges associated with LLMs inclu...,single_hop_specifc_query_synthesizer
2,Wut were the key advancemints in AI during 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,what google doin with them llms and how it aff...,[easy to follow. The rest of the document incl...,A leaked Google document titled 'We Have No Mo...,single_hop_specifc_query_synthesizer
4,How does the black box nature of Large Languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
5,How does the black box nature of Large Languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
6,How does the black box nature of Large Languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
7,How has OpenAI contributed to the development ...,[<1-hop>\n\nCode may be the best application T...,OpenAI was one of the first organizations to r...,multi_hop_abstract_query_synthesizer
8,How does the environmental impact of training ...,[<1-hop>\n\neasy to follow. The rest of the do...,The environmental impact of training large lan...,multi_hop_specific_query_synthesizer
9,How has the development of Claude 3 and its su...,[<1-hop>\n\nPrompt driven app generation is a ...,The development of Claude 3 and its subsequent...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [19]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [20]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [21]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [23]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [24]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [26]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [28]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [29]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents refer to AI systems that can act on your behalf, but the term is considered vague and lacks a clear definition. There are two main perspectives on what agents entail: one sees them as entities that perform tasks like a travel agent, while another views them as large language models (LLMs) that use tools to solve problems. The concept of autonomy is also associated with agents, but again, without a clear definition. Overall, the term "agents" reflects ongoing discussions and excitement, despite a lack of concrete examples or implementations in production.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [30]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [31]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
We are assessing if the answer is correct, using our reference output as a comparison.

- `labeled_helpfulness_evaluator`:
We are assessing if the answer is helpful to the user, in accordance with the reference answer

- `dope_or_nope_evaluator`:
We are assessing if the answer is dope/cool or not

## LangSmith Evaluation

In [32]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'crazy-end-48' at:
https://smith.langchain.com/o/de6b5317-873e-4f7c-aa6e-e780c215cc5d/datasets/1be3de1d-ecbd-4bcf-89f3-f76dea462548/compare?selectedSessions=50dc2f8e-e5a4-4e38-8d2c-e9b4b175fdf3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"In 2023, what were the significant advancement...","In 2023, significant advancements in Large Lan...",None,"In 2023, significant advancements in Large Lan...",1,0,0,2.990302,e09c60b0-cadd-4147-90f6-5d431c69762d,7fab01bb-c8c7-48e0-bf81-54689c34946c
1,How does the cost efficiency of GPT-4o compare...,"GPT-4o is priced at $2.50 per million tokens, ...",None,GPT-4o is significantly more cost-efficient co...,1,1,0,3.770814,0677513d-f287-4a9d-8c9f-27db8d93645f,963f893e-ada9-4856-a547-b84f566f6238
2,How has the development of Claude 3 and its su...,I don't know.,None,The development of Claude 3 and its subsequent...,0,0,0,0.631786,4c96ad3c-1395-4008-9b67-10616e39192e,dd34c187-ba74-4e34-bdf4-a34d7dbfa5d5
3,How does the environmental impact of training ...,The context mentions that the environmental im...,None,The environmental impact of training large lan...,1,1,0,3.987092,0e464901-5efc-4e67-b905-46dbdbb9ec63,972652b6-f122-45bd-81a5-29083733bc0c
4,How has OpenAI contributed to the development ...,OpenAI has significantly contributed to the de...,None,OpenAI was one of the first organizations to r...,1,0,0,3.457167,1ac88348-56dc-46a0-af61-5d095751ff7f,21d5e6ca-3324-43fd-9210-6dfd8f8ea220
5,How does the black box nature of Large Languag...,The black box nature of Large Language Models ...,None,The black box nature of Large Language Models ...,1,0,0,2.696270,e958eddd-a63c-44e6-8ed8-7beeaba0bb46,e6589f4b-a2dd-4542-ad38-1b25867970df
6,How does the black box nature of Large Languag...,I don't know.,None,The black box nature of Large Language Models ...,0,0,0,4.475883,c023174e-18fe-47cb-bb53-8e58f67b3630,f3b66ac0-b45e-4696-ab3a-9256b4e749c8
7,How does the black box nature of Large Languag...,I don't know.,None,The black box nature of Large Language Models ...,0,0,0,1.036467,cc4d0f87-34bd-4744-bb92-0d051c257319,b65da846-4561-444a-a94e-f6810f817245
8,what google doin with them llms and how it aff...,I don't know.,None,A leaked Google document titled 'We Have No Mo...,0,0,0,0.709054,a37ee6d6-1427-4609-aab0-2442a2b14bb9,4ebcbd8a-a86b-4cb2-bf89-9cbeaa242426
9,Wut were the key advancemints in AI during 2023?,The key advancements in AI during 2023 include...,None,2023 was the breakthrough year for Large Langu...,1,1,1,3.064141,4ac6be9c-f7d3-43ab-ac06-15a63e80b34f,a752ae9c-57bc-481c-85e1-bbcd2a404104


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [43]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [44]:
rag_documents = docs

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

### Answer #2:

** Accuracy ** \n
This depends on the data and the question. Large chunk size though can provide more context, it can bring in unncessary data.
Also vice-versa is true.

** Latency **
Large chunk size means more computation and storage is required and hence the latency will increase.

In [36]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### Answer #3:

** Accuracy **
Accuracy of the answer will likely be increased on larger embedding model since it can understand the data, context and question much better

** Latency **
Larger embedding model means higher dimensions and lot more number of variables our data is represented by.
On a query, larger embedding model means exponentially greater matrix multiplications and hence the latency will be increased.

In [37]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [38]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents, huh? Well, they’re this super vague term floating around in the AI scene. Some folks think of them as digital homies that act on your behalf, like a travel agent, while others see them as AI models, like LLMs, that can run tools in a loop to solve problems. But here’s the kicker: the term is kinda like a mixed-up smoothie without a clear recipe. So, if someone tells you they’re building "agents," you might just be scratching your head, \'cause it could mean a whole bunch of things. They\'re still in the "coming soon" phase and really depend on overcoming some serious gullibility issues. So, yeah, agents are a bit of a mystery for now!'

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'tart-meeting-1' at:
https://smith.langchain.com/o/de6b5317-873e-4f7c-aa6e-e780c215cc5d/datasets/1be3de1d-ecbd-4bcf-89f3-f76dea462548/compare?selectedSessions=c16f6cd0-5e50-495c-84d5-573dbab7b273




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"In 2023, what were the significant advancement...","Oh, you wanna talk about the major breakthroug...",None,"In 2023, significant advancements in Large Lan...",1,1,1,4.625615,e09c60b0-cadd-4147-90f6-5d431c69762d,262e67bf-5e2a-4cb5-8439-e0acd06b97dd
1,How does the cost efficiency of GPT-4o compare...,"Yo, check it out! GPT-4o is a total game chang...",None,GPT-4o is significantly more cost-efficient co...,1,1,1,6.757615,0677513d-f287-4a9d-8c9f-27db8d93645f,a2907b4d-9d16-4ff0-bab6-84f2cb3252f9
2,How has the development of Claude 3 and its su...,"Yo, the rise of Claude 3 and its offshoots, li...",None,The development of Claude 3 and its subsequent...,1,0,1,4.206849,4c96ad3c-1395-4008-9b67-10616e39192e,8eca625f-c64d-4ffa-abd7-b321d08a8909
3,How does the environmental impact of training ...,"Yo, check this out! The environmental impact o...",None,The environmental impact of training large lan...,1,1,1,5.323209,0e464901-5efc-4e67-b905-46dbdbb9ec63,c42271a7-fe8f-4ee1-9852-752daf69c898
4,How has OpenAI contributed to the development ...,"Yo, OpenAI's been a major player in the LLM ga...",None,OpenAI was one of the first organizations to r...,1,0,1,2.951929,1ac88348-56dc-46a0-af61-5d095751ff7f,e2ea30e2-0959-4008-8587-99bc3058d36f
5,How does the black box nature of Large Languag...,"Alright, listen up! The black box nature of LL...",None,The black box nature of Large Language Models ...,1,0,1,4.612774,e958eddd-a63c-44e6-8ed8-7beeaba0bb46,8f31a3e1-dc3c-49eb-9eb7-b77d21bb8365
6,How does the black box nature of Large Languag...,I don't know.,None,The black box nature of Large Language Models ...,0,0,0,0.787278,c023174e-18fe-47cb-bb53-8e58f67b3630,2b24a82f-8c3b-4155-bc54-36af507fd798
7,How does the black box nature of Large Languag...,I don't know.,None,The black box nature of Large Language Models ...,0,0,0,1.421352,cc4d0f87-34bd-4744-bb92-0d051c257319,892fce1c-b17a-4981-9889-bc062e0dc9d4
8,what google doin with them llms and how it aff...,I don't know.,None,A leaked Google document titled 'We Have No Mo...,0,0,0,1.004857,a37ee6d6-1427-4609-aab0-2442a2b14bb9,8330bc3e-88dd-441c-bd2b-6320d03d368d
9,Wut were the key advancemints in AI during 2023?,"Yo, 2023 was a wild ride for AI, especially wi...",None,2023 was the breakthrough year for Large Langu...,1,1,1,4.198389,4ac6be9c-f7d3-43ab-ac06-15a63e80b34f,6202365e-b3a4-4693-b261-669761032af2


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### Answer #3:

![comparision of runs](./compare.png)

1) The dopeness increased dramatically due to the prompt. We asked the model to give more cool responses.
2) P50 and P90 latencies of the dopeness example increase since its using a large embedding model
3) total tokens reduced because of large chunk size.



